# Laboratoire 2
## Gyroscopes à taux (rate gyroscope)

**Matériel nécessaire fourni** : table tournante, surface antidérapante,
règle de 30 cm, masking tape, LEGOs inclinés.

Ce laboratoire utilise le gyro embarqué du robot Kobuki. Il s'agit du STMicroelectronics [L3G4200D](http://www.st.com/content/ccc/resource/technical/document/datasheet/04/46/d6/00/be/d9/46/ae/CD00265057.pdf/files/CD00265057.pdf/jcr:content/translations/en.CD00265057.pdf).

![img](img/gyro_schema.png)
![img](img/gyro_datasheet.png)

### Partie 1 - Familiarisation avec le capteur

In [ ]:
# Importation des modules

%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from robmob.robot import Robot
from robmob.sensors import GyroSensor
from robmob.visualization import Visualizer

Modifiez la valeur de `ip_robot` selon le robot qui vous a été assigné.

In [ ]:
# Connexion au robot
ip_robot = '192.168.0.100'
robot = Robot(ip_robot)
robot.connect()

Il faut ensuite ajouter le gyroscope aux capteurs écoutés par le robot.

In [ ]:
gyro = GyroSensor()
robot.add_sensor(gyro)

En guise de *sanity check*, le code suivant affiche la dernière mesure du gyro. Assurez-vous que le robot est immobile avant de prendre la mesure.

In [ ]:
gyro.peek_data()

Vous devriez remarquer qu'il y a passablement de bruit dans le capteur. Maintenant, installer le robot sur un plateau rotatif et faites le tourner à différentes vitesses. Remarquez le comportement de la mesure en z. Comment se comporte-t-elle à des vitesses de rotation plus élevées?

Le code suivant affiche la mesure en temps réel. Appuyer deux fois sur la touche *i* ou appuyez sur le stop dans le haut de la page pour arrêter la boucle.

In [ ]:
import time
from IPython.display import clear_output

while True:
    mesure = gyro.peek_data()
    clear_output(wait=True)
    print("x: %0.10f deg/sec" % mesure['x'])
    print("y: %0.10f deg/sec" % mesure['y'])
    print("z: %0.10f deg/sec" % mesure['z'])
    time.sleep(0.05)

> **NOTE** Selon la [documentation](http://kobuki.yujinrobot.com/wiki/gyro-details/) de la plateforme Kobuki, le gyroscope est utilisé en mode 250 dps.

### Partie 2 - Calibration du gyroscope
Selon la datasheet, la valeur de DVoff, ou *digital zero-rate level*, se situe entre -10 et 10 dps. Autrement dit, lorsque la plateforme est immobile, la mesure ne sera pas zéro, mais se situera entre -10 et +10 degrés par seconde.


Pour cette partie, laissez le robot immobile durant 30 secondes.
>**Attention!** Accrocher, faire vibrer (ou même souffler!) sur le capteur faussera le résultat.


In [ ]:
samples = gyro.sample_data_for_x_sec(30) # samples a 3 colonnes: x, y et z
z_samples = samples[:, 2]                # notation numpy pour toutes les lignes, colonne 2
print("Dimension de z_samples: ", z_samples.shape)

Puisque le roboto se déplace sur un environnement 2D, nous utiliserons uniquement l'axe des z (voir le schéma au haut du notebook).

In [ ]:
z_offset = np.mean(z_samples)
z_offset

Cette valeur `z_offset` permettra d'ajuster les mesures dans les parties suivantes.

### Partie 3 - Intégration des mesures

Dans cette partie, nous allons écrire un programme qui donne l'angle du robot en degré en intégrant les mesures du gyro. Pour ce faire, prenez notes que la frequence d'aquisition des données est 108Hz, tel que stocké dans la constante gyro.SAMPLE_RATE. 

#### Exercice
Écrivez une fonction qui prend en entrée une mesure du gyro, intègre la mesure angulaire et l'accumule dans une variable `current_angle`. N'oubliez pas de tenir compte du biais `z_offset`.

> **PROTIP** Utilisez l'opérateur modulo % pour ramener l'angle en 0 et 360

In [ ]:
current_angle = 0

def integrate_gyro_measure(measure):
    global current_angle
    # modifiez current_angle de sorte qu'il intègre la nouvelle mesure
    # chaque fois que cette fonction est appelée.

In [ ]:
# Le programme suivant met à jour current_angle en appelant votre fonction à une fréquence de 108Hz

import time
from IPython.display import clear_output

sleep_time_sec = 1.0 / gyro.SAMPLE_RATE

def show_robot_angle():
    while True:
        measure = gyro.peek_data()['z']  # read_data() retire la donnée lue, peak_data() la laisse dans le buffer
        integrate_gyro_measure(measure)
    
        clear_output(wait=True)
        print(current_angle)
        time.sleep(sleep_time_sec)
    
show_robot_angle()

Pour tester l'intégration des mesures, placez le robot sur un plateau de verre rotatif. Fixez un bout de papier comme point de repère sur le robot et placez le robot au centre de 4 tuiles de plancher (les joints de tuiles représentant 0 deg, 90 deg, 180 deg et 270 deg). Faites tourner le robot et observez les mesures.

> **PROTIP** Il est possible que vous observiez que l'angle dérive rapidement. Ce la peut être causé par un changement de température du robot (voir OffDr dans la datasheet). Si cela est le cas, réévaluez le code de la partie 2 pour recalculer `z_offset`

### Partie 4 - Influence de l'angle du gyroscope

Faites de nouveaux la partie précédente, mais en plaçant le capteur sur un plan
incliné (utilisez les Lego inclinés à cette fin). Faites quelques tours de 360 degrés, toujours en marquant
un tour complet d’une pause de quelques secondes. Comparez les résultats de l'angle final obtenu avec
l'intégration du signal calibré par rapport à un estimé de 360 degrés. En particulier, est-ce
capteur vient changer la mesure?

### Partie 5 -  Estimation de la dérive du gyroscope

#### Exercice
Orientez le robot dans un angle connu à l'aide d'une référence. Faites une capture d’environ une
minute en vous inspirant du code de la partie 2, pendant laquelle vous faites des rotations horaires et antihoraires de
10-100 degrés autour de cette référence, donc sans jamais compléter un tour. Ceci simulera un robot
qui se déplace en tournant régulièrement, par exemple, mais qui ne fait jamais un tour complet. Avant
la fin de l’enregistrement, revenez exactement à l’angle de référence du départ, et conservez le robot
dans cet angle.
Une fois l’enregistrement terminé, faites l'intégration du signal calibré. S’il n’y avait pas
d’erreur, l’angle du début devrait être le même que celui de la fin. Quelle est l’erreur accumulée
(dérive) sur l’angle que vous constatez?

In [ ]:
# Votre code ici

### Partie 6 – Création d’une carte de l’environnement

Vous allez maintenant faire une carte de l'environnement en utilisant le capteur infrarouge et le gyroscope. La carte sera construite en faisant tourner le robot. Ainsi, vous allez
scanner l’environnement en 2D, avec des mesures en coordonnées polaires (i.e. distance et angle).
Déposez la table tournante et le robot sur le plancher. Assurez-vous d’avoir des obstacles intéressants
(boites de carton, poubelle ronde, jambes stables, mur, etc) autour du robot. Assurez-vous également
que tous les obstacles soient situés à 20 cm ou plus du robot (le capteur IR fonctionne pour des
distances de 20-150 cm). Pour tracer cette carte, l'angle du robot est calculé en fonction du temps.Amusez-vous à faire plusieurs cartes en déplaçant les objets entre les scans! Il se peut que des
objets plats apparaissent bombés : ceci sera le symptôme attribuable à la calibration en distance du
capteur infrarouge par défaut dans le script qui est différente de votre capteur.

In [ ]:
# Aquisition de 1000 points de la carte à environ 10Hz

import time, math
from robmob.sensors import SharpSensor
from scipy.interpolate import interp1d

# Listes des points de la carte à remplir
map_points = []

# Ajout du capteur sharp
ir_sensor = SharpSensor(1)
robot.add_sensor(ir_sensor)

current_angle = 0
last_measure_time = time.time()

fn_volt_to_cm = interp1d(ir_sensor.HIGH_RANGE_CALIB_TABLE[:,1], ir_sensor.HIGH_RANGE_CALIB_TABLE[:,0])

for i in range(200):
    time.sleep(0.1)
    
    gyro_measures_z = gyro.peek_buffer()[:, 2]
    time_since_last_measure, last_measure_time = time.time() - last_measure_time, time.time()
    number_of_measures = math.floor(time_since_last_measure * gyro.SAMPLE_RATE)
    
    #Intégration des dernière mesures depuis la dernière mise-à-jour
    current_angle += np.sum((gyro_measures_z[-number_of_measures:] - z_offset) * (1/gyro.SAMPLE_RATE))
    
    current_ir_volt = ir_sensor.peek_data()
    
    map_points.append([math.radians(current_angle), fn_volt_to_cm(current_ir_volt)])

In [ ]:
# Affichage de la carte
import matplotlib.pyplot as plt
points = np.asarray(map_points)

plt.axes(polar = True)
plt.scatter(points[:,0], points[:,1])